# 第8章: ニューラルネット

第7章で取り組んだポジネガ分類を題材として、ニューラルネットワークで分類モデルを実装する。なお、この章ではPyTorchやTensorFlow、JAXなどの深層学習フレームワークを活用せよ。

## 70. 単語埋め込みの読み込み

事前学習済み単語埋め込みを活用し、$|V| \times d_\rm{emb}$ の単語埋め込み行列$\pmb{E}$を作成せよ。ここで、$|V|$は単語埋め込みの語彙数、$d_\rm{emb}$は単語埋め込みの次元数である。ただし、単語埋め込み行列の先頭の行ベクトル$\pmb{E}_{0,:}$は、将来的にパディング（`<PAD>`）トークンの埋め込みベクトルとして用いたいので、ゼロベクトルとして予約せよ。ゆえに、$\pmb{E}$の2行目以降に事前学習済み単語埋め込みを読み込むことになる。

もし、Google Newsデータセットの[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)（300万単語・フレーズ、300次元）を全て読み込んだ場合、$|V|=3000001, d_\rm{emb}=300$になるはずである（ただ、300万単語の中には、殆ど用いられない稀な単語も含まれるので、語彙を削減した方がメモリの節約になる）。

また、単語埋め込み行列の構築と同時に、単語埋め込み行列の各行のインデックス番号（トークンID）と、単語（トークン）への双方向の対応付けを保持せよ。

In [2]:
import numpy as np
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format(
    "data\GoogleNews-vectors-negative300.bin",
    binary=True,
)

vocab_size = len(model.key_to_index) + 1
embedding_dim = model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim), dtype=np.float32)

token2id = {"<pad>": 0}
id2token = {0: "<pad>"}

for idx, word in enumerate(model.key_to_index):
    token2id[word] = idx + 1
    id2token[idx + 1] = word
    embedding_matrix[idx + 1] = model[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (3000001, 300)


## 71. データセットの読み込み

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) をダウンロードし、訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、全てのテキストをトークンID列に変換せよ。このとき、単語埋め込みの語彙でカバーされていない単語は無視し、トークン列に含めないことにせよ。また、テキストの全トークンが単語埋め込みの語彙に含まれておらず、空のトークン列となってしまう事例は、訓練セットおよび開発セットから削除せよ（このため、第7章の実験で得られた正解率と比較できなくなることに注意せよ）。

事例の表現方法は任意でよいが、例えば"contains no wit , only labored gags"がネガティブに分類される事例は、次のような辞書オブジェクトで表現すればよい。

```
{'text': 'contains no wit , only labored gags',
 'label': tensor([0.]),
 'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])}
```

この例では、`text`はテキスト、`label`は分類ラベル（ポジティブなら`tensor([1.])`、ネガティブなら`tensor([0.])`）、`input_ids`はテキストのトークン列をID列で表現している。

In [3]:
import os
import zipfile
import requests

# データセットのダウンロード
url = "https://dl.fbaipublicfiles.com/glue/data/SST-2.zip"
zip_path = "./data/SST-2.zip"

response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)
    
# ZIPファイルを解凍
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("./data")

In [7]:
import pandas as pd
import torch

train_path = "./data/SST-2/train.tsv"
dev_path = "./data/SST-2/dev.tsv"

def load_data(filepath):
    df = pd.read_csv(filepath, sep="\t", header=0)
    df.columns = ["text", "label"]
    df["label"] = df["label"].astype(int)
    return df

def make_dict(df):
    dataset = []
    for _, row in df.iterrows():
        text = row["text"]
        label = row["label"]
        
        # テキストをトークン化し、IDに変換
        words = text.split()
        input_ids = []
        
        for word in words:
            if word in token2id:
                input_ids.append(token2id[word])
        
        # 空のトークン列の事例は無視する
        if len(input_ids) > 0:
            dataset.append({
                "text": text,
                "label": torch.tensor([float(label)]),
                "input_ids": torch.tensor(input_ids)
            })
    
    return dataset

# データの読み込みと処理
train_df = load_data(train_path)
dev_df = load_data(dev_path)

train_dataset = make_dict(train_df)
dev_dataset = make_dict(dev_df)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Dev dataset size: {len(dev_dataset)}")
print("\nExample:")
print(train_dataset[0])
   


Train dataset size: 66650
Dev dataset size: 872

Example of processed data:
{'text': 'hide new secretions from the parental units ', 'label': tensor([0.]), 'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])}


## 72. Bag of wordsモデルの構築

単語埋め込みの平均ベクトルでテキストの特徴ベクトルを表現し、重みベクトルとの内積でポジティブ及びネガティブを分類するニューラルネットワーク（ロジスティック回帰モデル）を設計せよ。

In [14]:
import torch
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, embedding_matrix, freeze_embedding=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)
        
    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        mean_embedded = embedded.mean(dim=0)
        return self.linear(mean_embedded)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LogisticRegression(embedding_matrix, True).to(device)

## 73. モデルの学習

問題72で設計したモデルの重みベクトルを訓練セット上で学習せよ。ただし、学習中は単語埋め込み行列の値を固定せよ（単語埋め込み行列のファインチューニングは行わない）。また、学習時に損失値を表示するなど、学習の進捗状況をモニタリングできるようにせよ。

In [20]:
class LogisticRegression(nn.Module):
    def __init__(self, embedding_matrix, freeze_embedding=True):
        super().__init__()
        # 事前学習済みの埋め込み行列から nn.Embedding を作成
        # freeze=True なら重みを学習させない（凍結）
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=freeze_embedding,
        )
        # 線形層：入力は埋め込みの次元数 → 出力は1
        self.linear = nn.Linear(embedding_matrix.shape[1], 1)

    def forward(self, input_ids):
        # 入力（トークンID列）を埋め込みベクトルに変換
        embedded = self.embedding(input_ids)  # [seq_len, emb_dim]
        # 埋め込みベクトルの平均を計算
        mean_embedded = embedded.mean(dim=0).unsqueeze(0)  # [1, emb_dim]
        return self.linear(mean_embedded)  # [1, 1]

model = LogisticRegression(embedding_matrix, True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

def train(model, dataset, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for data in dataset:
        input_ids = data["input_ids"].to(device)
        labels = data["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        # ラベルを [1] → [1, 1] に変形して損失を計算
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataset)

num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train(model, train_dataset, optimizer, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}")

torch.save(model.state_dict(), "./model/No_73.pt")

Epoch 1/5, Loss: 0.6528
Epoch 2/5, Loss: 0.5941
Epoch 3/5, Loss: 0.5532
Epoch 4/5, Loss: 0.5239
Epoch 5/5, Loss: 0.5021


## 74. モデルの評価

問題73で学習したモデルの開発セットにおける正解率を求めよ。

In [26]:

import numpy as np
from sklearn.metrics import accuracy_score

def evaluate(model, dataset, device):
    model.eval() 

    y_preds = []  
    y_trues = [] 

    for data in dataset:
        input_ids = data["input_ids"].to(device)
        labels = data["label"].to(device)

        with torch.no_grad():  # 勾配計算を無効化（高速化・メモリ削減）
            outputs = model(input_ids)  
            probs = (
                torch.sigmoid(outputs).squeeze().cpu().numpy()
            )  # 確率に変換

            y_preds.append(probs)
            y_trues.append(labels.item())  # .item() で整数に変換

    y_preds = np.array(y_preds)
    y_trues = np.array(y_trues)

    binary_preds = (y_preds > 0.5).astype(int)
    accuracy = accuracy_score(y_trues, binary_preds)

    return accuracy

accuracy = evaluate(model, dev_dataset, device)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.7615


## 75. パディング

複数の事例が与えられたとき、これらをまとめて一つのテンソル・オブジェクトで表現する関数`collate`を実装せよ。与えられた複数の事例のトークン列の長さが異なるときは、トークン列の長さが最も長いものに揃え、0番のトークンIDでパディングをせよ。さらに、トークン列の長さが長いものから順に、事例を並び替えよ。

例えば、訓練データセットの冒頭の4事例が次のように表されているとき、

```
[{'text': 'hide new secretions from the parental units',
  'label': tensor([0.]),
  'input_ids': tensor([  5785,     66, 113845,     18,     12,  15095,   1594])},
 {'text': 'contains no wit , only labored gags',
  'label': tensor([0.]),
  'input_ids': tensor([ 3475,    87, 15888,    90, 27695, 42637])},
 {'text': 'that loves its characters and communicates something rather beautiful about human nature',
  'label': tensor([1.]),
  'input_ids': tensor([    4,  5053,    45,  3305, 31647,   348,   904,  2815,    47,  1276,  1964])},
 {'text': 'remains utterly satisfied to remain the same throughout',
  'label': tensor([0.]),
  'input_ids': tensor([  987, 14528,  4941,   873,    12,   208,   898])}]
```

`collate`関数を通した結果は以下のようになることが想定される。

```
{'input_ids': tensor([
    [     4,   5053,     45,   3305,  31647,    348,    904,   2815,     47,   1276,   1964],
    [  5785,     66, 113845,     18,     12,  15095,   1594,      0,      0,      0,      0],
    [   987,  14528,   4941,    873,     12,    208,    898,      0,      0,      0,      0],
    [  3475,     87,  15888,     90,  27695,  42637,      0,      0,      0,      0,      0]]),
 'label': tensor([
    [1.],
    [0.],
    [0.],
    [0.]])}
```


## 76. ミニバッチ学習

問題75のパディングの処理を活用して、ミニバッチでモデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 77. GPU上での学習

問題76のモデル学習をGPU上で実行せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 78. 単語埋め込みのファインチューニング

問題77の学習において、単語埋め込みのパラメータも同時に更新するファインチューニングを導入せよ。また、学習したモデルの開発セットにおける正解率を求めよ。

## 79. アーキテクチャの変更

ニューラルネットワークのアーキテクチャを自由に変更し、モデルを学習せよ。また、学習したモデルの開発セットにおける正解率を求めよ。例えば、テキストの特徴ベクトル（単語埋め込みの平均ベクトル）に対して多層のニューラルネットワークを通したり、畳み込みニューラルネットワーク（CNN; Convolutional Neural Network）や再帰型ニューラルネットワーク（RNN; Recurrent Neural Network）などのモデルの学習に挑戦するとよい。